<a href="https://colab.research.google.com/github/Sujal3141/Sujal/blob/main/House%20price%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
!mkdir ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [3]:
!cp kaggle.json ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip house-prices-advanced-regression-techniques

Archive:  house-prices-advanced-regression-techniques.zip
replace data_description.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data_description.txt    
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [38]:
import pandas as pd
import matplotlib as plt
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [7]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [8]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [9]:
#simple imputer
trf1 = ColumnTransformer([('imputed_LotFrontage',SimpleImputer(),[3]),
                          ('imputed_Alley',SimpleImputer(strategy = 'most frequent'),[6]),
                          ('imputed_MasVnrType',SimpleImputer(strategy = 'most_frequent'),[25]),
                          ('imputed_MasVnrArea',SimpleImputer(),[26]),
                          ('imputed_BsmtQual',SimpleImputer(strategy='most_frequent'),[30]),
                          ('imputed_BsmtCond',SimpleImputer(strategy='most_frequent'),[31]),
                          ('imputed_BsmtExposure',SimpleImputer(strategy='most_frequent'),[32]),
                          ('imputed_BsmtFinType1',SimpleImputer(strategy='most_frequent'),[33]),
                          ('imputed_BsmtFinType2',SimpleImputer(strategy='most_frequent'),[35]),
                          ('imputed_Electrical',SimpleImputer(strategy='most_frequent'),[42]),
                          ('imputed_FireplaceQu',SimpleImputer(strategy='most_frequent'),[57]),
                          ('imputed_GarageType',SimpleImputer(strategy='most_frequent'),[60]),
                          ('imputed_GarageQual',SimpleImputer(strategy='most_frequent'),[63]),
                          ('imputed_GarageCond',SimpleImputer(strategy='most_frequent'),[64]),
                          ('imputed_PoolQC',SimpleImputer(strategy='most_frequent'),[72]),
                          ('imputed_Fence',SimpleImputer(strategy='most_frequent'),[73]),
                          ('imputed_MiscFeature',SimpleImputer(strategy='most_frequent'),[74]),


                      ],remainder = 'passthrough')

In [34]:
#Making a list of columns which are of datatypes objects
object_dt = train.select_dtypes(include='object')

In [36]:
list_object_dt = list(object_dt)


In [37]:
#vARIABLE ENCODING
trf2 = ColumnTransformer([('encoding',pd.get_dummies(trf1,columns=list_object_dt))],remainder='passthrough')








